# MPS Import (Renormalizer)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/block-hczhai/block2-preview/blob/master/docs/source/tutorial/import_renormalizer_mps.ipynb)

In [1]:
!pip install block2==0.5.4rc5 -qq --progress-bar off --extra-index-url=https://block-hczhai.github.io/block2-preview/pypi/
!pip install pyscf==2.8.0 -qq --progress-bar off
!pip install renormalizer==0.0.11 -qq --progress-bar off

In [2]:
# update pyblock2/algebra/io.py
!wget https://raw.githubusercontent.com/jiangtong1000/block2-preview/refs/heads/io/pyblock2/algebra/io.py \
  -O /usr/local/lib/python3.11/dist-packages/pyblock2/algebra/io.py

--2025-06-14 15:19:57--  https://raw.githubusercontent.com/jiangtong1000/block2-preview/refs/heads/io/pyblock2/algebra/io.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45369 (44K) [text/plain]
Saving to: ‘/usr/local/lib/python3.11/dist-packages/pyblock2/algebra/io.py’

/usr/local/lib/pyth 100%[===================>]  44.31K  --.-KB/s    in 0.01s   

2025-06-14 15:19:57 (3.89 MB/s) - ‘/usr/local/lib/python3.11/dist-packages/pyblock2/algebra/io.py’ saved [45369/45369]



Sometimes one may need to handle the overlap between a good DMRG solution from ``block2`` and some external dense MPS, eg. Gaussian MPS that is useful for quantum information or in QMC calculations (https://arxiv.org/abs/2405.05440), or MPS from other DMRG codes, eg. Renormalizer (https://github.com/shuaigroup/Renormalizer).

In [3]:
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2.algebra.io import MPSTools
import numpy

bond_dims = [20] * 4 + [30] * 4
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-10] * 8

from pyscf import gto, tools, lo, scf
r0 = 3.0
mol = gto.M(atom=f'H 0 0 0; H 0 0 {r0}; H 0 0 {r0*2}; H 0 0 {r0*3}; H 0 0 {r0*4}; H 0 0 {r0*5}', basis='sto-3g', unit='Bohr')
mol.build()

mf = scf.RHF(mol)
mo_coeff = lo.orth.lowdin(mf.mol.intor('cint1e_ovlp_sph'))
mf.mo_coeff = mo_coeff
ncas, n_elec, spin, ecore, h1e, h2e, orb_sym = itg.get_rhf_integrals(mf, g2e_symm=8)

# Run DMRG calculation with Sz symmetry
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)
mpo = driver.get_qc_mpo(h1e=h1e, g2e=h2e, ecore=ecore, iprint=1)
zket = driver.get_random_mps(tag="KETSZ", bond_dim=20, nroots=1)
energies = driver.dmrg(mpo, zket, n_sweeps=20, bond_dims=bond_dims, noises=noises, thrds=thrds, iprint=1)
zket = driver.adjust_mps(zket, dot=1)[0]
driver.align_mps_center(zket, ref=0)
print('DMRG calculation with Sz symmetry finished, energies:', energies)

# this will be used to fill the external MPS's tensors
pyzket = MPSTools.from_block2(zket)

integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =       2286

Build MPO | Nsites =     6 | Nterms =       2286 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     6 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.001 T = 0.018
 Site =     1 /     6 .. Mmpo =    66 DW = 0.00e+00 NNZ =      243 SPT = 0.8584 Tmvc = 0.001 T = 0.015
 Site =     2 /     6 .. Mmpo =   110 DW = 0.00e+00 NNZ =      459 SPT = 0.9368 Tmvc = 0.001 T = 0.014
 Site =     3 /     6 .. Mmpo =    66 DW = 0.00e+00 NNZ =     1147 SPT = 0.8420 Tmvc = 0.003 T = 0.023
 Site =     4 /     6 .. Mmpo =    26 DW = 0.00e+00 NNZ =      243 SPT = 0.8584 Tmvc = 0.000 T = 0.015
 Site =     5 /     6 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.006
Ttotal =      0.090 Tmvc-total = 0.006 MPO bond dimension =   110 MaxDW = 0.00e+00
NNZ =         2144 SIZE =        18004 SPT = 0.8809

Rank =     0 Ttotal =      0.173 MPO method = FastBipartite bond dime

Here we will run a DMRG calculation with SU2 symmetry, and later we also compute the overlap between the external MPS and DMRG ground state.

In [4]:
# Run DMRG calculation with SU2 symmetry
driver_SU2 = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2)
driver_SU2.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)
mpo = driver_SU2.get_qc_mpo(h1e=h1e, g2e=h2e, ecore=ecore, iprint=1)
ket = driver_SU2.get_random_mps(tag="KETSU2", bond_dim=20, nroots=1)
energies = driver_SU2.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises, thrds=thrds, iprint=1)
ket = driver_SU2.adjust_mps(ket, dot=1)[0]
driver_SU2.align_mps_center(ket, ref=0)
print('DMRG calculation with SU2 symmetry finished, energies:', energies)

integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =        863

Build MPO | Nsites =     6 | Nterms =        863 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     6 .. Mmpo =    13 DW = 0.00e+00 NNZ =       13 SPT = 0.0000 Tmvc = 0.000 T = 0.009
 Site =     1 /     6 .. Mmpo =    34 DW = 0.00e+00 NNZ =       97 SPT = 0.7805 Tmvc = 0.000 T = 0.013
 Site =     2 /     6 .. Mmpo =    56 DW = 0.00e+00 NNZ =      180 SPT = 0.9055 Tmvc = 0.000 T = 0.013
 Site =     3 /     6 .. Mmpo =    34 DW = 0.00e+00 NNZ =      405 SPT = 0.7873 Tmvc = 0.000 T = 0.010
 Site =     4 /     6 .. Mmpo =    14 DW = 0.00e+00 NNZ =      100 SPT = 0.7899 Tmvc = 0.000 T = 0.011
 Site =     5 /     6 .. Mmpo =     1 DW = 0.00e+00 NNZ =       14 SPT = 0.0000 Tmvc = 0.000 T = 0.004
Ttotal =      0.059 Tmvc-total = 0.002 MPO bond dimension =    56 MaxDW = 0.00e+00
NNZ =          809 SIZE =         4753 SPT = 0.8298

Rank =     0 Ttotal =      0.104 MPO method = FastBipartite bond dime

Renormalizer runs DMRG with Sz symmetry by default, and the MPS object is a dense MPS. In the following we an external MPS using Renormalizer as an example.

In [10]:
import tempfile
from renormalizer import Model, Mps, Mpo, optimize_mps
from renormalizer.model import h_qc
from renormalizer.utils import log
from logging import INFO

logger = log.logging.getLogger("renormalizer")
logger.setLevel(INFO)

numpy.product = numpy.prod # a little bit hacky, since renormalizer's code is using an older numpy.

ftmp = tempfile.NamedTemporaryFile()
tools.fcidump.from_mo(mol, ftmp.name, mo_coeff)
h1e_spin, h2e_spin, nuc = h_qc.read_fcidump(ftmp.name, ncas)
basis, ham_terms = h_qc.qc_model(h1e_spin, h2e_spin)
model = Model(basis, ham_terms)
mpo = Mpo(model)
print(f"mpo_bond_dims:{mpo.bond_dims}")

energy_list = {}
M = 30
procedure = [[M, 0.4], [M, 0.2], [M, 0.1], [M, 0], [M, 0], [M,0], [M,0]]
mps = Mps.random(model, mol.nelec, M, percent=1.0)
mps.optimize_config.procedure = procedure
mps.optimize_config.method = "2site"
energies, mps = optimize_mps(mps.copy(), mpo)
gs_e = min(energies)+nuc
print(f'lowest energy from Renormalizer MPS: {gs_e}')

2025-06-14 15:21:18,040[INFO] nuclear repulsion: 2.899999999999999
INFO:renormalizer.model.h_qc:nuclear repulsion: 2.899999999999999
2025-06-14 15:21:18,046[INFO] spin norbs: 12
INFO:renormalizer.model.h_qc:spin norbs: 12
2025-06-14 15:21:19,820[INFO] optimization method: 2site
INFO:renormalizer.mps.gs:optimization method: 2site
2025-06-14 15:21:19,823[INFO] e_rtol: 1e-06
INFO:renormalizer.mps.gs:e_rtol: 1e-06
2025-06-14 15:21:19,826[INFO] e_atol: 1e-08
INFO:renormalizer.mps.gs:e_atol: 1e-08
2025-06-14 15:21:19,829[INFO] procedure: [[30, 0.4], [30, 0.2], [30, 0.1], [30, 0], [30, 0], [30, 0], [30, 0]]
INFO:renormalizer.mps.gs:procedure: [[30, 0.4], [30, 0.2], [30, 0.1], [30, 0], [30, 0], [30, 0], [30, 0]]


mpo_bond_dims:[1, 4, 16, 37, 54, 68, 81, 68, 54, 37, 16, 4, 1]


2025-06-14 15:21:24,823[INFO] DMRG has converged!
INFO:renormalizer.mps.gs:DMRG has converged!
2025-06-14 15:21:24,895[INFO] mps current size: 48.4KiB, Matrix product bond dim:[1, 2, 4, 8, 16, 30, 30, 30, 16, 8, 4, 2, 1]
INFO:renormalizer.mps.gs:mps current size: 48.4KiB, Matrix product bond dim:[1, 2, 4, 8, 16, 30, 30, 30, 16, 8, 4, 2, 1]


lowest energy from Renormalizer MPS: -2.9576460390051147


Note that now we obtain a dense MPS in the spin orbital basis, to make it compatible with ``block2``, we need to convert it to a fermion orbital basis. We also need to make the MPS left canonical, for the step after.

In [6]:
mps.ensure_left_canonical()

def spin_to_fermion_mps(mps_spin, qn):
    nsites_spin = len(mps_spin)
    assert nsites_spin % 2 == 0
    nsites_fermion = nsites_spin // 2
    qnl_fermion = []
    mps_fermion = []

    for i in range(nsites_fermion):
        ml = mps_spin[2*i].shape[0]
        mr = mps_spin[2*i+1].shape[-1]
        merged_array = numpy.einsum("ipj, jql->ipql", mps_spin[2*i], mps_spin[2*i+1])
        merged_array = numpy.transpose(merged_array, (0, 2, 1, 3))
        merged_array = merged_array.reshape(ml, 4, mr)
        mps_fermion.append(merged_array)
        qnl_fermion.append(qn[2*i])
    qnl_fermion.append(qn[-1])
    return mps_fermion, qnl_fermion

qn = []
for i_mps in range(len(mps)):
    iqn = [x.tolist() for x in mps.qn[i_mps]]
    qn.append(iqn)
qn[0] = [[0, 0]]
qn.append([[mol.nelec[0], mol.nelec[1]]])

mps_fermion, qn_fermion = spin_to_fermion_mps(mps, qn)
mps_list = []
for i_mps in range(len(mps_fermion)):
    mps_list.append(mps_fermion[i_mps])

Now we have a list of dense MPS tensors in the fermion orbital basis, and a list of their quantum numbers (from left to right). And we are ready to transform them to the ``block2`` format.

In [7]:
print('bond dimensions:', [imps.shape[0] for imps in mps_fermion])
print('quantum numbers:', qn_fermion)

mps_in_block2 = MPSTools.trans_renormalizer_sz_to_block2_sz(mps_list, qn_fermion)

bond dimensions: [1, 4, 16, 30, 16, 4]
quantum numbers: [[[0, 0]], [[1, 0], [0, 1], [1, 1], [0, 0]], [[0, 1], [0, 1], [1, 2], [1, 2], [2, 1], [2, 1], [0, 0], [1, 1], [1, 1], [1, 1], [1, 1], [2, 0], [0, 2], [2, 2], [1, 0], [1, 0]], [[0, 1], [1, 2], [1, 2], [1, 2], [1, 2], [2, 1], [2, 1], [2, 1], [2, 1], [3, 1], [3, 1], [1, 1], [1, 1], [1, 1], [1, 1], [0, 3], [2, 0], [2, 0], [3, 0], [2, 3], [0, 2], [0, 2], [2, 2], [2, 2], [2, 2], [2, 2], [1, 0], [3, 2], [1, 3], [1, 3]], [[1, 2], [1, 2], [2, 1], [2, 1], [3, 1], [1, 1], [2, 3], [2, 3], [3, 3], [2, 2], [2, 2], [2, 2], [2, 2], [3, 2], [3, 2], [1, 3]], [[2, 3], [3, 3], [2, 2], [3, 2]], [[3, 3]]]


Here ``mps_in_block2`` contains all the tensors in the pyblock2 format, we can further convert it to block2 format.

In [8]:
driver.symm_type = SymmetryTypes.SZ
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin)
impo = driver.get_identity_mpo()
for it, ts in enumerate(pyzket.tensors):
    pyzket.tensors[it] = mps_in_block2[it]
bra = MPSTools.to_block2(pyzket, driver.basis, tag='bra_sz')
print('overlap between Renormalizer Sz MPS with block2 Sz MPS:', driver.expectation(bra, impo, zket))

overlap between Renormalizer Sz MPS with block2 Sz MPS: -0.9999999659563918


We may also compute the overlap between the Renormalizer Sz MPS and the block2 SU2 MPS. To do this, we first convert the Renormalizer Sz MPS to a SU2 MPS.

In [9]:
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin)
for it, ts in enumerate(pyzket.tensors):
    pyzket.tensors[it] = mps_in_block2[it]
bra = MPSTools.trans_sz_to_su2(pyzket, driver.basis, zket.info.target, target_twos=0)

driver.symm_type = SymmetryTypes.SU2
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin)
bra = MPSTools.to_block2(bra, driver.basis, tag='bra_su2')

impo = driver.get_identity_mpo()
print('overlap between Renormalizer Sz MPS with block2 SU2 MPS:', driver.expectation(bra, impo, ket))

overlap between Renormalizer Sz MPS with block2 SU2 MPS: -0.9999990300114634


Note that different DMRG runs may lead to sign differences.